<a href="https://colab.research.google.com/github/jhenningsen/Equity_Analysis/blob/main/LangStudio/SMA_Model_Backtest_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import os

# Point to your Drive folder
drive_lib = '/content/drive/My Drive/python_libs'
if drive_lib not in sys.path:
    sys.path.insert(0, drive_lib)

try:
    import pandas_ta as ta
    import pandas as pd
    import numpy as np
    print("Success! Environment is ready for 2022 analysis.")
    print(f"Pandas Version: {pd.__version__}")
except Exception as e:
    print(f"Error: {e}")

Success! Environment is ready for 2022 analysis.
Pandas Version: 2.2.2


In [2]:
import pandas as pd
import yfinance as yf
import pandas_ta as ta
import numpy as np

# 1. Configuration
SMA_RANGE = range(3, 21)
BB_PERIODS = [10, 20, 30]
SYMBOLS = ["TSLA", "SPY", "QQQ", "NVDA", "META", "MSTR", "COIN", "GLD", "AMD", "SLV", "PLTR", "MSFT", "ORCL", "IWM", "AAPL", "AVGO", "AMZN", "UNH", "NFLX", "MU", "GOOGL", "TSM", "LULU", "CRWV", "GOOG", "IBIT", "JPM", "HOOD", "GDX", "ADBE", "NOW", "APP", "GS", "WOLF", "BABA", "IREN", "COST", "INTC", "LLY", "CRCL", "CVNA", "SNDK", "OKLO", "SMH", "BA", "BMNR", "ASTS", "NBIS", "SOFI", "BE"]

# 2. Optimized Data Cache
print("Fetching fresh data...")
data_cache = {}
for s in SYMBOLS:
    df = yf.download(s, period="5y", interval="1d", progress=False, auto_adjust=True)
    #df = yf.download(s, start="2021-12-01", end="2023-01-01", interval="1d", progress=False, auto_adjust=True)
    if not df.empty:
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)
        data_cache[s] = df

# 3. Generate Raw Trades with Technical Audit Columns
all_trades_list = []
print("Generating Technical Audit Logs...")

for sma_val in SMA_RANGE:
    for bb_val in BB_PERIODS:
        for symbol, df_orig in data_cache.items():
            df = df_orig.copy()

            # Indicators
            df['SMA_Value'] = ta.sma(df['Close'], length=sma_val)
            bb = ta.bbands(df['Close'], length=bb_val, std=2)
            if bb is None: continue

            bbm_col = [c for c in bb.columns if c.startswith('BBM')][0]
            df['BB_Midpoint'] = bb[bbm_col]

            # Logic: Cross above SMA AND Below BB Midpoint
            df['Prev_Close'] = df['Close'].shift(1)
            df['Prev_SMA'] = df['SMA_Value'].shift(1)

            cond_cross = (df['Close'] > df['SMA_Value']) & (df['Prev_Close'] <= df['Prev_SMA'])
            cond_value = df['Close'] < df['BB_Midpoint']

            # Filter and Label
            trades = df[cond_cross & cond_value].copy()
            if not trades.empty:
                # Forward Returns
                trades['Ret_3D'] = df['Close'].pct_change(3).shift(-3)
                trades['Ret_5D'] = df['Close'].pct_change(5).shift(-5)
                trades['Ret_10D'] = df['Close'].pct_change(10).shift(-10)

                # Metadata
                trades['Symbol'] = symbol
                trades['SMA_Param'] = sma_val
                trades['BB_Param'] = bb_val

                # Selecting Audit Columns
                audit_cols = [
                    'Symbol', 'SMA_Param', 'BB_Param',
                    'Close', 'SMA_Value', 'BB_Midpoint',
                    'Ret_3D', 'Ret_5D', 'Ret_10D'
                ]
                all_trades_list.append(trades[audit_cols])

master_trade_log = pd.concat(all_trades_list)
master_trade_log = master_trade_log.reset_index() # Keep the Date column
print(f"Done! {len(master_trade_log)} total trade instances recorded.")

# VIEW SPECIFIC SYMBOL DETAIL (Example: NVDA)
display(master_trade_log[master_trade_log['Symbol'] == 'NVDA'].head(20))

Fetching fresh data...


ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['CRWV']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-12-01 -> 2023-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1638334800, endDate = 1672549200")')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['IBIT']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-12-01 -> 2023-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1638334800, endDate = 1672549200")')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['WOLF']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-12-01 -> 2023-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1638334800, endDate = 1672549200")')
ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['CRCL']: YFPricesMissingError('possibly delisted; no price data found  (1d 2021-12-01 -> 2023-01-01) (Yahoo error = "Data doesn\'t exist for startDate = 1638334800, endDate = 1672549200")')
ERROR:yfinance:
1 Failed download:
E

Generating Technical Audit Logs...
Done! 23015 total trade instances recorded.


Price,Date,Symbol,SMA_Param,BB_Param,Close,SMA_Value,BB_Midpoint,Ret_3D,Ret_5D,Ret_10D
79,2021-12-15,NVDA,3,10,30.400108,28.929626,30.416480,-0.089957,-0.034768,-0.028661
80,2021-12-21,NVDA,3,10,29.018787,28.143816,29.188759,0.064316,0.031849,-0.050593
81,2022-01-11,NVDA,3,10,27.763216,27.434853,28.610079,-0.031456,-0.098860,-0.181364
82,2022-01-28,NVDA,3,10,22.795845,22.475131,23.822653,0.105166,0.064755,0.048555
83,2022-02-24,NVDA,3,10,23.702084,23.130192,24.424686,-0.011411,-0.001262,-0.045736
84,2022-03-09,NVDA,3,10,22.973419,21.921275,23.209069,-0.073173,0.064395,0.113844
85,2022-04-04,NVDA,3,10,27.311760,27.071516,27.344001,-0.115205,-0.198940,-0.188670
86,2022-04-13,NVDA,3,10,22.163847,21.836096,24.421563,-0.000225,-0.090979,-0.109039
87,2022-04-18,NVDA,3,10,21.744591,21.709651,23.327794,-0.073452,-0.086352,-0.103291
88,2022-04-25,NVDA,3,10,19.866909,19.831635,21.157128,-0.006030,-0.018541,-0.148327


In [3]:
# 1. Grouping by Symbol and Strategy Parameters
performance_summary = master_trade_log.groupby(['Symbol', 'SMA_Param', 'BB_Param']).agg(
    Trade_Count=('Ret_3D', 'count'),
    Wins_3D=('Ret_3D', lambda x: (x > 0).sum()),
    Wins_5D=('Ret_5D', lambda x: (x > 0).sum()),
    Wins_10D=('Ret_10D', lambda x: (x > 0).sum()),
    Avg_3D_Ret=('Ret_3D', 'mean'),
    Avg_5D_Ret=('Ret_5D', 'mean'),
    Avg_10D_Ret=('Ret_10D', 'mean')
).reset_index()

# 2. Calculate Win Rates
performance_summary['Win_Rate_3D'] = performance_summary['Wins_3D'] / performance_summary['Trade_Count']
performance_summary['Win_Rate_5D'] = performance_summary['Wins_5D'] / performance_summary['Trade_Count']
performance_summary['Win_Rate_10D'] = performance_summary['Wins_10D'] / performance_summary['Trade_Count']

# Define display columns for consistency
display_cols = [
    'Symbol', 'SMA_Param', 'BB_Param', 'Trade_Count',
    'Win_Rate_3D', 'Avg_3D_Ret',
    'Win_Rate_5D', 'Avg_5D_Ret',
    'Win_Rate_10D', 'Avg_10D_Ret'
]

# --- SECTION A: Best Parameter Set PER Symbol ---
# Sort by Symbol and Win Rate, then take the top 1 for each ticker
best_per_symbol = performance_summary.sort_values(
    ['Symbol', 'Win_Rate_10D', 'Avg_10D_Ret'],
    ascending=[True, False, False]
).groupby('Symbol').head(1)

print("--- [A] BEST PARAMETER COMBO PER SYMBOL (Top 25 by Win Rate) ---")
display(best_per_symbol[display_cols].sort_values('Win_Rate_10D', ascending=False).head(25).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))


--- [A] BEST PARAMETER COMBO PER SYMBOL (Top 25 by Win Rate) ---


,Symbol,SMA_Param,BB_Param,Trade_Count,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
28,AAPL,13,10,1,100.0%,4.24%,100.0%,3.69%,100.0%,1.12%
99,ADBE,19,20,1,100.0%,0.85%,100.0%,7.30%,100.0%,7.61%
203,AMZN,20,10,1,100.0%,3.04%,100.0%,11.20%,100.0%,9.57%
415,BABA,9,10,1,100.0%,4.48%,100.0%,6.99%,100.0%,29.21%
249,APP,18,20,2,50.0%,1.29%,50.0%,-1.54%,100.0%,4.83%
299,ASTS,18,10,1,100.0%,40.89%,100.0%,23.41%,100.0%,51.41%
332,AVGO,13,10,1,100.0%,7.52%,100.0%,5.07%,100.0%,5.01%
464,BE,9,10,2,100.0%,6.55%,100.0%,11.43%,100.0%,18.22%
743,GLD,19,20,1,100.0%,2.08%,100.0%,1.72%,100.0%,3.16%
675,GDX,14,10,1,0.0%,-2.37%,0.0%,-4.70%,100.0%,4.62%


In [4]:
# 1. Global Aggregation: Using the Raw Trade Log
# This avoids 'average of averages' for both Win Rates AND Returns.
global_strategy_raw = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Symbol': 'count',      # Total Trade Count
    'Ret_3D': [
        ('Wins', lambda x: (x > 0).sum()),
        ('Avg_Ret', 'mean')
    ],
    'Ret_5D': [
        ('Wins', lambda x: (x > 0).sum()),
        ('Avg_Ret', 'mean')
    ],
    'Ret_10D': [
        ('Wins', lambda x: (x > 0).sum()),
        ('Avg_Ret', 'mean')
    ]
})

# 2. Flatten Multi-index columns for cleaner access
global_strategy_raw.columns = [
    'Total_Trades',
    'Wins_3D', 'Avg_3D_Ret',
    'Wins_5D', 'Avg_5D_Ret',
    'Wins_10D', 'Avg_10D_Ret'
]
global_strategy_raw = global_strategy_raw.reset_index()

# 3. Recalculate Global Win Rates from raw counts
global_strategy_raw['Win_Rate_3D'] = global_strategy_raw['Wins_3D'] / global_strategy_raw['Total_Trades']
global_strategy_raw['Win_Rate_5D'] = global_strategy_raw['Wins_5D'] / global_strategy_raw['Total_Trades']
global_strategy_raw['Win_Rate_10D'] = global_strategy_raw['Wins_10D'] / global_strategy_raw['Total_Trades']

# 4. Final Formatting & Sorting
final_cols = [
    'SMA_Param', 'BB_Param', 'Total_Trades',
    'Win_Rate_3D', 'Avg_3D_Ret',
    'Win_Rate_5D', 'Avg_5D_Ret',
    'Win_Rate_10D', 'Avg_10D_Ret'
]

global_strategy_raw = global_strategy_raw[final_cols].sort_values(by='Win_Rate_10D', ascending=False)

print("--- GLOBAL PARAMETER LEADERS (Recalculated from Raw Trade Data) ---")
display(global_strategy_raw.head(10).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))

--- GLOBAL PARAMETER LEADERS (Recalculated from Raw Trade Data) ---


,SMA_Param,BB_Param,Total_Trades,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
26,12,10,48,45.8%,0.05%,60.4%,1.83%,66.7%,1.23%
32,14,10,67,53.7%,1.35%,55.2%,1.47%,65.7%,3.03%
29,13,10,68,50.0%,0.85%,60.3%,1.47%,64.7%,1.87%
23,11,10,40,55.0%,1.33%,60.0%,1.86%,62.5%,-0.60%
35,15,10,73,41.1%,-0.31%,47.9%,-0.04%,54.8%,2.29%
38,16,10,75,37.3%,-0.69%,44.0%,-0.87%,45.3%,1.13%
25,11,30,517,45.3%,-0.96%,42.7%,-1.62%,43.7%,-1.82%
28,12,30,479,45.9%,-0.85%,44.3%,-1.40%,43.6%,-1.56%
34,14,30,423,48.0%,-0.64%,44.2%,-1.32%,43.5%,-1.67%
31,13,30,449,45.4%,-0.94%,44.3%,-1.61%,42.8%,-1.93%


In [7]:
# --- Symbol Drill-Down Section ---
target_symbol = "NVDA"  # Change this to any symbol from your list

print(f"--- Top 10 Parameter Combinations for {target_symbol} (Sorted by Avg_10D_Ret) ---")

# Filter the performance_summary we created above
symbol_drill_down = performance_summary[performance_summary['Symbol'] == target_symbol]

# Display the results
display(symbol_drill_down[display_cols].sort_values('Avg_10D_Ret', ascending=False).head(10).style.format({
    'Win_Rate_3D': '{:.1%}', 'Avg_3D_Ret': '{:.2%}',
    'Win_Rate_5D': '{:.1%}', 'Avg_5D_Ret': '{:.2%}',
    'Win_Rate_10D': '{:.1%}', 'Avg_10D_Ret': '{:.2%}'
}))

--- Top 10 Parameter Combinations for NVDA (Sorted by Avg_10D_Ret) ---


,Symbol,SMA_Param,BB_Param,Trade_Count,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
1570,NVDA,12,20,8,37.5%,-0.76%,50.0%,-0.39%,62.5%,4.10%
1582,NVDA,16,20,5,60.0%,3.01%,60.0%,1.91%,60.0%,4.07%
1576,NVDA,14,20,7,42.9%,0.23%,42.9%,-1.80%,57.1%,3.59%
1563,NVDA,9,30,11,45.5%,0.65%,45.5%,0.32%,45.5%,3.28%
1585,NVDA,17,20,3,66.7%,3.72%,66.7%,3.99%,66.7%,3.23%
1567,NVDA,11,20,9,33.3%,-1.52%,44.4%,-2.13%,55.6%,2.71%
1577,NVDA,14,30,10,50.0%,0.89%,50.0%,-0.98%,50.0%,2.64%
1562,NVDA,9,20,12,33.3%,-0.83%,33.3%,-1.56%,50.0%,2.47%
1560,NVDA,8,30,11,54.5%,1.06%,45.5%,-0.11%,45.5%,2.35%
1568,NVDA,11,30,10,40.0%,-1.66%,50.0%,-1.39%,50.0%,2.29%


In [8]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Prepare the Aggregate Data (if not already done in Cell 2)
param_perf = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Ret_3D': 'mean',
    'Ret_5D': 'mean',
    'Ret_10D': 'mean'
}).reset_index()

# 2. Define the horizons we want to visualize
horizons = [('3D', 'Ret_3D'), ('5D', 'Ret_5D'), ('10D', 'Ret_10D')]

print("--- PARAMETER SURFACE HEATMAPS (Average Returns) ---")

for label, col_name in horizons:
    # Pivot the data for the heatmap
    pivot_table = param_perf.pivot(index="SMA_Param", columns="BB_Param", values=col_name)

    # Create the visualization using Pandas Styler for a clean Colab output
    print(f"\n{label} Horizon Average Return:")
    display(pivot_table.style.background_gradient(cmap='RdYlGn', axis=None)
            .format("{:.2%}")
            .set_caption(f"Average {label} Return by SMA and BB Period"))

# Optional: If you prefer a more "visual" graphical heatmap using Seaborn:
# plt.figure(figsize=(10, 6))
# sns.heatmap(pivot_table, annot=True, fmt=".2%", cmap="RdYlGn")
# plt.title("10D Return Surface")
# plt.show()

--- PARAMETER SURFACE HEATMAPS (Average Returns) ---

3D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,-0.33%,-0.31%,-0.28%
4,-0.51%,-0.54%,-0.43%
5,-0.90%,-0.95%,-0.75%
6,-1.14%,-1.06%,-0.74%
7,-1.05%,-1.12%,-0.84%
8,-0.85%,-1.15%,-0.76%
9,-1.27%,-1.37%,-0.94%
10,nan%,-1.29%,-0.98%
11,1.33%,-1.15%,-0.96%



5D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,-1.01%,-1.05%,-0.94%
4,-1.19%,-1.20%,-0.96%
5,-1.62%,-1.67%,-1.22%
6,-2.11%,-1.98%,-1.42%
7,-2.19%,-2.17%,-1.71%
8,-2.52%,-2.38%,-1.70%
9,-2.76%,-2.35%,-1.87%
10,nan%,-2.13%,-1.76%
11,1.86%,-1.91%,-1.62%



10D Horizon Average Return:


BB_Param,10,20,30
SMA_Param,,,
3,-2.12%,-2.16%,-1.68%
4,-2.37%,-2.48%,-1.88%
5,-2.81%,-2.91%,-1.98%
6,-3.08%,-2.93%,-1.98%
7,-2.32%,-2.66%,-2.00%
8,-1.78%,-2.62%,-1.68%
9,-1.74%,-2.65%,-1.81%
10,nan%,-2.88%,-1.99%
11,-0.60%,-2.48%,-1.82%


In [9]:
# 1. Prepare the Win Rate Aggregate Data
# We calculate the mean of the boolean (Return > 0) to get the percentage of wins
win_rate_perf = master_trade_log.groupby(['SMA_Param', 'BB_Param']).agg({
    'Ret_3D': lambda x: (x > 0).mean(),
    'Ret_5D': lambda x: (x > 0).mean(),
    'Ret_10D': lambda x: (x > 0).mean()
}).reset_index()

# 2. Define the horizons for visualization
horizons = [('3-Day', 'Ret_3D'), ('5-Day', 'Ret_5D'), ('10-Day', 'Ret_10D')]

print("--- PARAMETER SURFACE HEATMAPS (Win Rate %) ---")
print("Focus: Probability of a positive return across the portfolio.")

for label, col_name in horizons:
    # Pivot the data for the heatmap
    pivot_win_rate = win_rate_perf.pivot(index="SMA_Param", columns="BB_Param", values=col_name)

    print(f"\n{label} Win Rate Surface:")
    # Using a 0.5 (50%) midpoint for the gradient helps highlight 'Edge' vs 'Coin Flip'
    display(pivot_win_rate.style.background_gradient(cmap='RdYlGn', vmin=0.45, vmax=0.65, axis=None)
            .format("{:.1%}")
            .set_caption(f"{label} Win Rate by SMA and BB Period"))

--- PARAMETER SURFACE HEATMAPS (Win Rate %) ---
Focus: Probability of a positive return across the portfolio.

3-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,44.8%,46.6%,47.2%
4,44.2%,45.9%,47.3%
5,40.1%,42.6%,44.5%
6,39.7%,42.1%,44.7%
7,41.2%,42.7%,45.2%
8,43.9%,44.3%,47.5%
9,42.3%,42.6%,46.2%
10,nan%,42.2%,45.5%
11,55.0%,42.3%,45.3%



5-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,42.3%,42.4%,44.1%
4,43.5%,43.5%,45.9%
5,39.4%,39.9%,43.4%
6,38.7%,39.9%,43.6%
7,38.5%,40.0%,42.2%
8,38.7%,40.3%,42.9%
9,37.2%,39.7%,42.0%
10,nan%,39.7%,41.5%
11,60.0%,41.1%,42.7%



10-Day Win Rate Surface:


BB_Param,10,20,30
SMA_Param,,,
3,39.2%,39.4%,41.0%
4,37.3%,37.5%,40.2%
5,35.6%,36.3%,39.9%
6,35.0%,36.7%,40.4%
7,36.8%,37.6%,40.0%
8,39.0%,38.2%,41.6%
9,39.4%,38.3%,41.7%
10,nan%,38.4%,41.7%
11,62.5%,41.6%,43.7%


In [10]:
# 1. Create a copy for export to keep the original data intact
export_df = global_strategy_raw.copy()

# 2. Round all numerical columns to 4 decimal places
# This ensures that Win Rates (0.6543 -> 0.65) and Returns (0.0123 -> 0.01) are consistent
export_df = export_df.round(4)

# 3. Rename columns for the final report
export_df = export_df.rename(columns={
    'SMA_Param': 'SMA',
    'BB_Param': 'BB',
    'Total_Trades': 'Trades'
})

# 4. Export to CSV string
# float_format="%.4f" is a safety measure to ensure trailing zeros are kept (e.g., 0.50)
csv_output = export_df.to_csv(index=False, float_format="%.4f")

print("--- COPY AND PASTE THIS TO GEMINI ---")
print(csv_output)

--- COPY AND PASTE THIS TO GEMINI ---
SMA,BB,Trades,Win_Rate_3D,Avg_3D_Ret,Win_Rate_5D,Avg_5D_Ret,Win_Rate_10D,Avg_10D_Ret
12,10,48,0.4583,0.0005,0.6042,0.0183,0.6667,0.0123
14,10,67,0.5373,0.0135,0.5522,0.0147,0.6567,0.0303
13,10,68,0.5000,0.0085,0.6029,0.0147,0.6471,0.0187
11,10,40,0.5500,0.0133,0.6000,0.0186,0.6250,-0.0060
15,10,73,0.4110,-0.0031,0.4795,-0.0004,0.5479,0.0229
16,10,75,0.3733,-0.0069,0.4400,-0.0087,0.4533,0.0113
11,30,517,0.4526,-0.0096,0.4275,-0.0162,0.4371,-0.0182
12,30,479,0.4593,-0.0085,0.4426,-0.0140,0.4363,-0.0156
14,30,423,0.4799,-0.0064,0.4421,-0.0132,0.4350,-0.0167
13,30,449,0.4543,-0.0094,0.4432,-0.0161,0.4276,-0.0193
17,10,81,0.3086,-0.0093,0.4198,-0.0116,0.4198,0.0040
12,20,385,0.4260,-0.0098,0.4234,-0.0169,0.4182,-0.0228
18,10,79,0.3291,-0.0037,0.3924,-0.0068,0.4177,0.0090
14,20,312,0.4519,-0.0076,0.4135,-0.0172,0.4167,-0.0251
9,30,576,0.4618,-0.0094,0.4201,-0.0187,0.4167,-0.0181
10,30,545,0.4550,-0.0098,0.4147,-0.0176,0.4165,-0.0199
11,20,435,0.4230,-0.0

In [11]:
# 1. Aggregate Advanced Risk Metrics from Raw Data
# We focus on the 10-Day window as it showed the highest potential for spreads
stress_test = master_trade_log.groupby(['SMA_Param', 'BB_Param'])['Ret_10D'].agg([
    ('Trades', 'count'),
    ('Avg_Ret', 'mean'),
    ('Median_Ret', 'median'),
    ('Std_Dev', 'std'),
    ('Win_Rate', lambda x: (x > 0).mean())
]).reset_index()

# 2. Calculate Expectancy Score (Assuming 1:1 Risk/Reward Ratio)
# Formula: (WinRate * 1) - (LossRate * 1)
# A score of 0.10 means you expect to make $0.10 for every $1.00 risked over time.
stress_test['Expectancy_Score'] = (stress_test['Win_Rate'] * 2) - 1

# 3. Sort by Expectancy Score
stress_test = stress_test.sort_values('Expectancy_Score', ascending=False)

print("--- STRATEGY EXPECTANCY & STRESS TEST (10-Day Window) ---")
print("Focus: Finding the most 'reliable' parameters, not just the 'luckiest'.")

display(stress_test.head(10).style.format({
    'Avg_Ret': '{:.2%}',
    'Median_Ret': '{:.2%}',
    'Std_Dev': '{:.2%}',
    'Win_Rate': '{:.1%}',
    'Expectancy_Score': '{:.3f}'
}).background_gradient(subset=['Expectancy_Score'], cmap='Greens'))

--- STRATEGY EXPECTANCY & STRESS TEST (10-Day Window) ---
Focus: Finding the most 'reliable' parameters, not just the 'luckiest'.


,SMA_Param,BB_Param,Trades,Avg_Ret,Median_Ret,Std_Dev,Win_Rate,Expectancy_Score
26,12,10,47,1.23%,1.45%,11.17%,66.7%,0.333
32,14,10,67,3.03%,1.16%,10.87%,65.7%,0.313
29,13,10,67,1.87%,1.45%,11.93%,64.7%,0.294
23,11,10,39,-0.60%,1.50%,9.19%,62.5%,0.250
35,15,10,72,2.29%,0.64%,10.73%,54.8%,0.096
38,16,10,74,1.13%,-0.65%,11.65%,45.3%,-0.093
25,11,30,496,-1.82%,-0.87%,11.36%,43.7%,-0.126
28,12,30,468,-1.56%,-0.92%,10.88%,43.6%,-0.127
34,14,30,415,-1.67%,-0.86%,10.35%,43.5%,-0.130
31,13,30,440,-1.93%,-1.02%,10.75%,42.8%,-0.145
